In [7]:
import pandas as pd
import math
import sys

In [8]:
path = 'D:\\C\\docs\\DTU\\2. Second sem\\ML\\Lab\\5'
filename = 'Dataset'
filename+= '.csv'
df = pd.read_csv(path + '\\' + filename)
df

,WorkPeriod,Income,Married,Rank,Buy House
0,Short,High,No,High,No
1,Short,High,No,High,No
2,Average,Medium,Yes,High,Yes
3,Long,Low,Yes,High,Yes
4,Long,Low,Yes,Low,Yes
5,Long,Low,No,High,No
6,Average,Low,No,Low,Yes
7,Short,Medium,Yes,Low,No
8,Short,Low,No,Low,Yes
9,Long,Medium,Yes,High,Yes


In [9]:
TARGET = df.columns[len(df.columns)-1]
TARGET

'Buy House'

Find target values in a dataframe

In [10]:
def find_target(data) :
    # columns of dataframe
    cols = list(data.columns)

    # target values for the column
    tgt_val = cols[len(cols)-1]
    target = set(data[tgt_val].tolist())

    return tgt_val, target

Calculating entropy of S(Entire Dataframe)

In [11]:
def find_S(data) :
    tgt_val, target = find_target(data)

    S = 0
    count = {}

    for i in target :
        count[i]=0
    tgt = data[tgt_val].tolist()

    for i in tgt:
        count[i]+=1

    for val in count :
        tmp = count[val]/len(tgt)
        S-= tmp * math.log2(tmp)
    return S

Entropy for entire Dataset(For target values)

In [12]:
S = find_S(df)
S

0.9967916319816366

Unique values in a column

In [13]:
def val_count(data, col, val) :
    return data.index[data[col]==val]

Unique values for every target

In [14]:
def tgt_count(data, col, val) :
    
    val_idx = val_count(data, col, val)
    temp = {}

    tgt_val, target = find_target(data)

    for i in target:
        temp[i]=0

    for idx in val_idx:
        temp[data.loc[idx][tgt_val]]+=1
        
    return temp

Function to find Entropy for a column value

In [15]:
def entropy(data, col, val) :
    temp = tgt_count(data, col, val)
    
    n = 0
    for i in temp:
        if temp[i]==0 :
            return 0
        n+=temp[i]
    
    ans = 0
    for i in temp:
        t = temp[i]/n
        ans-= t * math.log2(t)
    return ans

Function to find gain for a column

In [16]:
def gain(data, col) :
    ans = S

    tmp = set(data[col].tolist())

    tgt_val, target = find_target(data)

    for val in tmp:    
        col_val = tgt_count(data, col, val) 
        col_val_count = sum(col_val.values())

        ans-= col_val_count/len(data[tgt_val]) * entropy(data, col, val)
    return ans

Calculating Split Info

In [17]:
def split_info(data, col) :
    val = []
    for i in set(df[col]) :
        val.append(i)
    
    ans = 0
    for i in val :
        tmp = len(val_count(df, col, i)) / df.shape[0]
        ans+= tmp * math.log2(tmp)

    return -ans

Gain Ratio

In [18]:
def gain_ratio(gain, split) :
    return gain / split

Function for finding Max gain in a column

In [19]:
def get_max_gain_ratio_feature(data) :
    cols = list(data.columns)
    features = cols[ : len(cols)-1]
    
    gn = gain(data, features[0])
    si = split_info(data, features[0])
    
    max_gain_ratio = gain_ratio(gn, si)

    max_gain_ratio_feature = features[0]

    for feature in features :
        gn = gain(data, feature)
        si = split_info(data, feature)
    
        gnr = gain_ratio(gn, si)
                
        if gnr > max_gain_ratio :
            max_gain_ratio = gnr
            max_gain_ratio_feature = feature

    return max_gain_ratio_feature

Node class for a Column value

In [20]:
class Node :
    def __init__(self, col_name=None, edge=None, df=None, tgt_val=None) :
        self.col_name = col_name
        self.df = df
        self.edge = edge
        self.tgt_val = tgt_val

Recursive function for generating Decision Tree

In [21]:
adj_list = []
indx = 0

def generate_tree(data, index) :
    if len(set(data[TARGET])) <= 1 :
        return

    col_name = get_max_gain_ratio_feature(data)
    
    for col in set(data[col_name]) :
        df = data[data[col_name] == col]

        temp = Node(col_name, col, df)
        adj_list.append(temp)
        
        tgt_val = 'Many'
        data_tmp = list(df[TARGET])
        if len(set(data_tmp)) == 1 :
            tgt_val = data_tmp[0]
        
        temp.tgt_val = tgt_val

        generate_tree(df, index+1)

In [22]:
generate_tree(df, 0)

Printing Node in DFS manner along with feature value and Target value

In [23]:
for nn in adj_list : 
    print('Feature Name : ', nn.col_name, ',     Feature Value : ', nn.edge, ",   Target Value : ", nn.tgt_val)

Feature Name :  Income ,     Feature Value :  Medium ,   Target Value :  Many
Feature Name :  Rank ,     Feature Value :  Low ,   Target Value :  No
Feature Name :  Rank ,     Feature Value :  High ,   Target Value :  Yes
Feature Name :  Income ,     Feature Value :  Low ,   Target Value :  Many
Feature Name :  Rank ,     Feature Value :  Low ,   Target Value :  Yes
Feature Name :  Rank ,     Feature Value :  High ,   Target Value :  Many
Feature Name :  Married ,     Feature Value :  No ,   Target Value :  No
Feature Name :  Married ,     Feature Value :  Yes ,   Target Value :  Yes
Feature Name :  Income ,     Feature Value :  High ,   Target Value :  No
